In [48]:
# import libraries

import pandas as pd
import numpy as np

In [49]:
# Load data 

train = pd.read_csv('/content/Train.csv')
test = pd.read_csv('/content/Test.csv')

In [50]:
train.shape,test.shape

((8068, 11), (2627, 10))

In [51]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [52]:
len(set(test['ID'].unique()).intersection(set(train['ID'].unique())))

2332

In [53]:
testx = pd.merge(test,train,how='left', on = 'ID')

In [54]:
test.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,458996,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6
3,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6
4,459001,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6


In [55]:
test['Var_1'].value_counts()

Cat_6    1672
Cat_4     386
Cat_3     267
Cat_2     141
Cat_7      66
Cat_1      34
Cat_5      29
Name: Var_1, dtype: int64

In [56]:
train['Var_1'].value_counts()

Cat_6    5238
Cat_4    1089
Cat_3     822
Cat_2     422
Cat_7     203
Cat_1     133
Cat_5      85
Name: Var_1, dtype: int64

In [57]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627 entries, 0 to 2626
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2627 non-null   int64  
 1   Gender           2627 non-null   object 
 2   Ever_Married     2577 non-null   object 
 3   Age              2627 non-null   int64  
 4   Graduated        2603 non-null   object 
 5   Profession       2589 non-null   object 
 6   Work_Experience  2358 non-null   float64
 7   Spending_Score   2627 non-null   object 
 8   Family_Size      2514 non-null   float64
 9   Var_1            2595 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 205.4+ KB


In [58]:
# check target distribution in train 

train['Segmentation'].value_counts()

D    2268
A    1972
C    1970
B    1858
Name: Segmentation, dtype: int64

In [59]:
train.columns

Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')

In [60]:
for i in ['Gender' , 'Ever_Married' , 'Graduated','Profession','Spending_Score','Var_1'  ]:
    print(f"column {i} unique values {train[i].unique()}")

column Gender unique values ['Male' 'Female']
column Ever_Married unique values ['No' 'Yes' nan]
column Graduated unique values ['No' 'Yes' nan]
column Profession unique values ['Healthcare' 'Engineer' 'Lawyer' 'Entertainment' 'Artist' 'Executive'
 'Doctor' 'Homemaker' 'Marketing' nan]
column Spending_Score unique values ['Low' 'Average' 'High']
column Var_1 unique values ['Cat_4' 'Cat_6' 'Cat_7' 'Cat_3' 'Cat_1' 'Cat_2' nan 'Cat_5']


In [61]:
# concatenate data into df ,  fill missing values with mode & label encode category values
train['is_train'] = 1
test['is_train'] = 0
df = pd.concat([train,test])

In [62]:

from sklearn.preprocessing import LabelEncoder

for i in ['Gender' , 'Ever_Married' , 'Graduated','Profession','Spending_Score','Var_1'  ]:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i].astype('str'))

In [63]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation,is_train
0,462809,1,0,22,0,5,1.0,2,4.0,3,D,1
1,462643,0,1,38,1,2,NaN,0,3.0,3,A,1
2,466315,0,1,67,1,2,1.0,2,1.0,5,B,1
3,461735,1,1,67,1,7,0.0,1,2.0,5,B,1
4,462669,0,1,40,1,3,NaN,1,6.0,5,A,1


In [64]:
df['Segmentation'] = df['Segmentation'].map({'A':0,'B':1,'C':2,'D':3})

In [65]:
train = df[df['is_train'] == 1]

In [66]:
pip install catboost

In [67]:
# split train into 5 folds and apply random forest and check accuracy of each fold

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold,KFold
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

x = train.drop(['Segmentation','is_train','ID'],axis=1)
y = train['Segmentation']
# rf = RandomForestClassifier(n_estimators=50)

categorical_features = ["Gender", "Ever_Married" ,"Graduated" ,"Profession" ,"Spending_Score" ,"Var_1" ]

# rf_1 = LGBMClassifier(learning_rate=0.1,
#                     boosting_type='gbdt', max_depth=4,  objective='multiclass', 
#                     random_state=100,  
#                   n_estimators=1000 ,reg_alpha=0, reg_lambda=1, n_jobs=-1)

# rf_2 = CatBoostClassifier(learning_rate=0.15, n_estimators=1000, subsample=0.085, max_depth=5, 
#                                                                  verbose=400,
#                                                                 bootstrap_type="Bernoulli",
#                                                                  class_weights=[1, 1, 1, 1],
#                                                                  loss_function='MultiClass')

# rf = VotingClassifier(estimators=[ 
#                                                                      ('CatBoost_Best', rf_1),
#                                                                      ('LightGBM_Best', rf_2),], 
#                                                                      voting='soft',weights=[5,5.2])

rf = LGBMClassifier(learning_rate=0.02,
                    boosting_type='gbdt', max_depth=4,  objective='multiclass', 
                    random_state=100,  
                  n_estimators=1000 ,reg_alpha=0, reg_lambda=1, n_jobs=-1)

# # all blank values replace with mode 
# for column in df.columns:
#     df[column].fillna(df[column].mode()[0], inplace=True)

kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=10)
acc = []

for fold,(t_id,v_id) in enumerate(kf.split(x,y)):
    tx = x.iloc[t_id]; ty = y.iloc[t_id]
    vx = x.iloc[v_id]; vy = y.iloc[v_id]
    rf.fit(tx,ty, 
           categorical_feature = categorical_features
#            cat_features = categorical_features
          )
           
    val_y = rf.predict(vx)
    acc_score = accuracy_score(vy,val_y)
    acc.append(acc_score)
    print(f"fold {fold} accuracy {acc_score}")

print(f"Mean accuracy score {np.mean(acc_score)}")

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Ever_Married', 'Gender', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


fold 0 accuracy 0.5415117719950434
fold 1 accuracy 0.5297397769516728
fold 2 accuracy 0.5223048327137546
fold 3 accuracy 0.5220086794792312
fold 4 accuracy 0.5610663360198388
Mean accuracy score 0.5610663360198388


In [68]:
x.columns

Index(['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1'],
      dtype='object')

In [69]:
sub = pd.read_csv('/content/sample_submission.csv')

In [70]:
test = df[df['is_train'] == 0][x.columns]
sub.Segmentation = rf.predict(test)

In [71]:
sub.head()

,ID,Segmentation
0,458989,0.0
1,458994,2.0
2,458996,0.0
3,459000,2.0
4,459001,3.0


In [72]:
sub.Segmentation = sub.Segmentation.map({0:'A', 1:'B' , 2:'C' ,3:'D'   })

In [73]:
sub.head()

,ID,Segmentation
0,458989,A
1,458994,C
2,458996,A
3,459000,C
4,459001,D


In [74]:
test.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,0,1,36,1,2,0.0,2,1.0,5
1,1,1,37,1,5,8.0,0,4.0,5
2,0,1,69,0,9,0.0,2,1.0,5
3,1,1,59,0,4,11.0,1,2.0,5
4,0,0,19,0,8,NaN,2,4.0,5


In [75]:
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation,is_train
0,462809,1,0,22,0,5,1.0,2,4.0,3,3.0,1
1,462643,0,1,38,1,2,NaN,0,3.0,3,0.0,1
2,466315,0,1,67,1,2,1.0,2,1.0,5,1.0,1
3,461735,1,1,67,1,7,0.0,1,2.0,5,1.0,1
4,462669,0,1,40,1,3,NaN,1,6.0,5,0.0,1


In [76]:
sub1 = sub

In [77]:
2332/2600


0.8969230769230769

In [78]:
sub1 = pd.merge(sub1,testx,how='left',on='ID')


In [79]:
sub1.head()


,ID,Segmentation_x,Gender_x,Ever_Married_x,Age_x,Graduated_x,Profession_x,Work_Experience_x,Spending_Score_x,Family_Size_x,Var_1_x,Gender_y,Ever_Married_y,Age_y,Graduated_y,Profession_y,Work_Experience_y,Spending_Score_y,Family_Size_y,Var_1_y,Segmentation_y
0,458989,A,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6,Female,Yes,42.0,Yes,Engineer,1.0,Low,1.0,Cat_6,B
1,458994,C,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6,Male,Yes,38.0,Yes,Healthcare,8.0,Average,4.0,Cat_6,C
2,458996,A,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6,Female,Yes,71.0,No,NaN,1.0,Low,1.0,Cat_6,A
3,459000,C,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6,Male,Yes,58.0,No,Executive,12.0,High,2.0,Cat_6,C
4,459001,D,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6,Female,No,20.0,No,Marketing,NaN,Low,4.0,Cat_6,C


In [80]:
sub['Segmentation2'] = sub1['Segmentation_y']


In [81]:
sub.head()


,ID,Segmentation,Segmentation2
0,458989,A,B
1,458994,C,C
2,458996,A,A
3,459000,C,C
4,459001,D,C


In [82]:
sub['Segmentation2'] = sub['Segmentation2'].fillna('x')


In [83]:
for i in range(len(sub)):
    if sub.iloc[i,2] != 'x':
        sub.iloc[i,1] = sub.iloc[i,2]

In [84]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627 entries, 0 to 2626
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             2627 non-null   int64 
 1   Segmentation   2627 non-null   object
 2   Segmentation2  2627 non-null   object
dtypes: int64(1), object(2)
memory usage: 61.7+ KB


In [85]:
sub[['ID','Segmentation']].to_csv('Last_Sub.csv',index = False)
